# Мастерская. Проект по предсказанию сердечного приступа.

В данном проекте нам предстоит нам предстоит разработать модель машинного обучния, а также подготовить библиотеку и интерфейс к ней для предсказания на тестовой выборке. 

В начале проекта нам доступен обучающий датасет и тестовый датасет, не содержащий правильных ответов. Метрика на тестовом датасете будет измерена в ходе проверки работы. Также нам доступен скрипт, с помощью которого будет измеряться финальная метрика вашей модели.

В данной работе мы сделаем следующие шаги:
- изучим предоставленные данные,
- проведем предобработку,
- исследования,
- объединение данных и выделение необходимых признаков,
- изучим взаимосвязь параметров между собой,
- построим модели,
- подобрем гипермараметры,
- выберем наиболее подходящую из них,
- подберем наиболее подходящую для проекта метрику,
- сделаем предсказание для тестовой выборки для отправки на проверку,
- кроме того, мы подготовим библиотеку и интерфейс к ней.

In [ ]:
#импорты и константы

import numpy as np
import pandas as pd
!pip install --upgrade matplotlib==3.5.0 -q
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st
!pip install --upgrade scikit-learn -q
from sklearn.model_selection import train_test_split

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder, 
    StandardScaler, 
    MinMaxScaler,
    LabelEncoder
)
from sklearn.compose import ColumnTransformer

# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем функцию для работы с метриками
from sklearn.metrics import roc_auc_score, make_scorer

# импортируем класс RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# загружаем нужные модели
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVC

!pip install phik -q
from phik import phik_matrix
from phik.report import plot_correlation_matrix


!pip install shap -q
import shap

RANDOM_STATE = 42
TEST_SIZE = 0.25

import warnings
#warnings.filterwarnings('ignore', category=FutureWarning)